In [3]:
# 1. 설치 (최초 1회만)
!pip install koeda

# 2. 라이브러리 불러오기
import pandas as pd
from koeda import EDA

# 3. 데이터 불러오기
url = "https://raw.githubusercontent.com/bth0114/AIFFEL_DLThon_DKTC_online13/main/data/raw_csv/merged_train.csv"
df = pd.read_csv(url)

# 4. 대상 클래스 필터링 후 클래스별 3개씩 샘플링
target_classes = ['기타 괴롭힘 대화', '협박 대화', '갈취 대화', '직장 내 괴롭힘 대화']
df_target = df[df['class'].isin(target_classes)]
df_sampled = df_target.groupby('class').apply(lambda x: x.sample(n=3, random_state=42)).reset_index(drop=True)

# 5. EDA 증강기 설정
eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.3, prob_rd=0.3)

# 6. 전처리 함수
def clean_text(text):
    return text.replace("\n", " ").replace("\r", "").strip()

# 7. 증강 수행
augmented_rows = []

for _, row in df_sampled.iterrows():
    text = row['conversation']
    label = row['class']
    
    try:
        eda_aug = eda(text, repetition=1)
        eda_aug = eda_aug if isinstance(eda_aug, list) else [eda_aug]
        aug_text = clean_text(eda_aug[0])
        if len(aug_text) > 5:
            augmented_rows.append({"conversation": aug_text, "class": label})
    except:
        pass

# 8. 결과 확인
df_augmented = pd.DataFrame(augmented_rows)
print(df_augmented)


                                         conversation        class
0   만 거기 예 부르셨너요 누구 왜부르시죠가 지갑을 잃어버려서 돈이없다 돈좀 있 아니요...        갈취 대화
1   당신 와이프 장해 이쁘더라 제발 저 사람 체통 매점 하고만 이야기 나눠주세요 깔끔하...        갈취 대화
2   저기요 아가씨 을 ? 손 치워요 야 새끼야 저 돈 거짓말이야 뒤지다 어쩔 아 진짜 ...        갈취 대화
3   또.자랑 . 자랑 사과 난 에게 도움이 정보를 주고 싶었던 것 .이라고.. 풋. 적...    기타 괴롭힘 대화
4   저기요 화채시킨 . 이 먹으라고 보낸건가요이 문제 화채 들어가는 이 커팅하는 모양도...    기타 괴롭힘 대화
5   도 멍청한 정도?같으니라고 야 왜그래 프린터가할머니제대로 안되잖아 내프사 카톡 됐지...    기타 괴롭힘 대화
6   오늘 가는거 알지 네 고 있습니다 저는 사정이 좀 있어서.사정? 사정이 왜 은 아니...  직장 내 괴롭힘 대화
7   사이 최 늑대 사원 잠깐 말미 만 인 이리 와줄 수 있어요 제명 ? 네! 바닥 대리...  직장 내 괴롭힘 대화
8   야 새끼야 내 앞에서 짭밥먹은 장 티 내지마 전선 내가 못하는것 처럼 보이잖아 아니...  직장 내 괴롭힘 대화
9   돼요 너 파 주소 빠지는거야아. 다 찾아가서 죽일거야 안까지?까지죄송합니다 뭐? 이...        협박 대화
10  뒤지고싶지? 시키는대로 안해? 죄송해요. 덜쳐맞았나봐? 납입가 사람 여럿 찔러봤다고...        협박 대화
11  이새끼 아직도 정신 못차려? 시키는대로만 ? 피흘리기 싫으면. 그 은 너무 난 못해...        협박 대화


In [4]:
# 1. 설치 (최초 1회만)
!pip install koeda

# 2. 라이브러리 불러오기
import pandas as pd
from koeda import EDA

# 3. 데이터 불러오기
url = "https://raw.githubusercontent.com/bth0114/AIFFEL_DLThon_DKTC_online13/main/data/raw_csv/merged_train.csv"
df = pd.read_csv(url)

# 4. EDA 증강기 설정 (기본값)
eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.5, alpha_ri=0.3, alpha_rs=0.3, prob_rd=0.3)

# 5. 전처리 함수
def clean_text(text):
    return text.replace("\n", " ").replace("\r", "").strip()

# 6. 전체 문장 EDA 수행
augmented_rows = []

for _, row in df.iterrows():
    text = row['conversation']
    label = row['class']
    
    try:
        eda_aug = eda(text, repetition=1)
        eda_aug = eda_aug if isinstance(eda_aug, list) else [eda_aug]
        aug_text = clean_text(eda_aug[0])
        if len(aug_text) > 5:
            augmented_rows.append({"conversation": aug_text, "class": label})
    except:
        pass

# 7. 증강 데이터프레임 만들기
df_augmented = pd.DataFrame(augmented_rows)

# 8. 원본 + 증강 병합
df_combined = pd.concat([df, df_augmented], ignore_index=True)
df_combined.drop_duplicates(subset=["conversation", "class"], inplace=True)
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# 9. 저장
df_combined.to_csv("final_train_allclass_eda.csv", index=False)

# 10. 클래스 분포 확인
print("클래스별 샘플 수:")
print(df_combined["class"].value_counts())


클래스별 샘플 수:
기타 괴롭힘 대화      2022
갈취 대화          1946
직장 내 괴롭힘 대화    1940
협박 대화          1784
일반 대화          1582
Name: class, dtype: int64


In [6]:
import os
print(os.getcwd())


/aiffel/AIFFEL_quest_review/Exploration/Ex01


In [7]:
from IPython.display import FileLink

# 파일 다운로드 링크 생성
FileLink('final_train_allclass_eda.csv')


/aiffel/AIFFEL_quest_review/Exploration/Ex01/final_train_allclass_eda.csv

In [8]:
# 이미 위에서 수행한 코드 기준으로 이어서 실행

# EDA로 생성된 데이터프레임만 저장
df_augmented.to_csv("eda_only.csv", index=False)

# 저장 확인용 출력
print("✅ EDA 증강본만 저장 완료!")
print("샘플 수:", len(df_augmented))
print("클래스별 분포:")
print(df_augmented["class"].value_counts())

✅ EDA 증강본만 저장 완료!
샘플 수: 4637
클래스별 분포:
기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           791
Name: class, dtype: int64


In [9]:
from IPython.display import FileLink

# 다운로드 가능한 링크 생성
FileLink('eda_only.csv')


/aiffel/AIFFEL_quest_review/Exploration/Ex01/eda_only.csv